## Step 1: Reading the data

In [1]:
import pyspark.sql.functions as F
import time

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [3]:
print(sc._conf.get('yarn.nodemanager.resource.memory-mb'))

None


In [4]:
# import pyspark
sc = spark.sparkContext
print('Original spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))
print('Original spark.executor.cores: ' + sc._conf.get('spark.executor.cores'))
print('Original spark.executor.memory: ' + sc._conf.get('spark.executor.memory'))
print('Original spark.app.name: ' + sc._conf.get('spark.app.name'))
print('Original spark.driver.memory: ' + sc._conf.get('spark.driver.memory'))
print(sc._conf.get('spark.cores.max'))

# #Stop existing Spark environment
# sc.stop()

# # Waiting for the environment to stop
# sleep_time = 10
# print(f'Waiting for {sleep_time} seconds for the enviroment to stop...')
# time.sleep(sleep_time)

# # Applying new configuration and restarting Spark
# conf = pyspark.SparkConf().setAll([('spark.app.name','Filtering Configs'),('spark.executor.memory', '12g'), ('spark.executor.cores', '12'),
#                                    ('spark.driver.memory','12g'),('spark.cores.max','12')])
# sc = pyspark.SparkContext(conf=conf)

# print('New spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))

# # Starting  Spark session with configs applied
# spark = SparkSession(sc).builder.getOrCreate()

Original spark.driver.maxResultSize: 1920m
Original spark.executor.cores: 2
Original spark.executor.memory: 5739m
Original spark.app.name: PySparkShell
Original spark.driver.memory: 3840m
None


In [ ]:
%%time

df_raw = spark.read.json('gs://msca-bdp-tweets/final_project')

23/03/07 23:19:48 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 5.07 s, sys: 1.1 s, total: 6.17 s
Wall time: 36min 14s


23/03/07 23:53:07 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Step 2: Filtering the unwanted tweets

Filering the tweets where the tweets are not liked, users with no followers as we need influentual users, tweets which aren't withheld in any country due to restrictions and only english tweets as we are more focused on english speaking users

In [ ]:
#test

In [ ]:
%%time

#     .filter('favourites_count > 0')\

df_raw_filtered = df_raw\
    .filter('user.location is not null')\
    .filter('user.followers_count > 0')\
    .filter('user.favourites_count > 0')\
    .filter('possibly_sensitive == FALSE or possibly_sensitive is NULL')\
    .filter('withheld_in_countries is NULL')\
    .filter('truncated == "False"')\
    .filter('lang == "en"')\
    .withColumn("tweet_text", F.lower(F.col("tweet_text")))\
    .withColumn('stripped', F.regexp_replace(F.col("tweet_text"),"[\;\@\$\#\,&%\"\.\']|(\b'\w)|(\w'\b)|https\S+|http\S+|www\.\S+|\\\\\S*",""))


print("Count of filtered tweets: ",df_raw_filtered.count())

Count of filtered tweets:  46263118
CPU times: user 3.88 s, sys: 845 ms, total: 4.72 s
Wall time: 25min 4s


### Step 2: Finding the keywords

Using the keywords generated by chatgpt

In [ ]:
# df_raw_filtered.limit(5)

In [ ]:
# education_keywords = [
#     'school','university','college','education','learning','students','teacher','curriculum',
#     'online','distance','stem','literacy','numeracy','assessment','tests','homework','study',
#     'success','graduation','vocational','adult','continuing','professional','preschool',
#     'elementary','middle','high','secondary','undergraduate','graduate','doctorate',
#     'mba','law','medical','nursing','engineering','business','art','music','language',
#     'trade','common','core','child','left','behind','every','student','succeeds','act',
#     'shortage','funding','management','special','gifted','english','learners','parent','involvement','safety',
#     'bullying','prevention','discipline','behavior','disabilities'
# ]

In [ ]:
# education_keywords = [
#     'elementary_school','elementary-school','elementary school','high school','university','college','education','learning','students','teacher','curriculum',
#     'stem','literacy','homework','graduation','vocational','preschool','books','k-12','k12',
#     'elementary','middle','secondary','undergraduate','graduate','doctorate',
#     'mba','law','engineering','student','succeeds','learners','parent','involvement','bullying','discipline'
# ]


In [ ]:
explicit_keywords = ['kill', 'killed', 'killing', 'die', 'dead', 'shoot', 'gun', 'murder', 'slaughter', 'shot', 'sex', 'fuck', 'porn', 'abortion', 'guns', 'dying', 
                     'terror', 'protest', 'violence', 'ukraine', 'crush', 'arrest', 'kidnap', 'victim', 'massacre', 'trans', 'lgbt']

In [ ]:
#Converting the list of keyword to regex
explicit_keywords_regex = '|'.join(["(" + c +")" for c in explicit_keywords])

In [ ]:
explicit_keywords_regex

'(kill)|(killed)|(killing)|(die)|(dead)|(shoot)|(gun)|(murder)|(slaughter)|(shot)|(sex)|(fuck)|(porn)|(abortion)|(guns)|(dying)|(terror)|(protest)|(violence)|(ukraine)|(crush)|(arrest)|(kidnap)|(victim)|(massacre)|(trans)|(lgbt)'

In [ ]:
#Converting the list of keyword to regex
education_keywords_regex

'(elementary school)|(elementaryschool)|(high school)|(university)|(college)|(education)|(learning)|(students)|(teacher)|(curriculum)|(stem)|(literacy)|(homework)|(graduation)|(preschool)|(k 12)|(k12)|(elementary)|(middle)|(secondary)|(undergraduate)|(graduate)|(doctorate)|(mba)|(law)|(engineering)'

## Step 3: Filtering the data based on the keywords and explicit keywords

In [ ]:
df_filtered_education = df_raw_filtered.where(F.col('stripped').rlike(education_keywords_regex))

In [20]:
df_filtered_education_explicit = df_filtered_education.filter(~F.col('stripped').rlike(explicit_keywords_regex))

In [21]:
df_filtered_education.count()

25395838

In [ ]:
df_filtered_education_explicit.count()

20623515

In [26]:
df_filtered_education_explicit.limit(1).collect()

[Row(coordinates=None, created_at='Wed May 25 21:22:47 +0000 2022', display_text_range=None, entities=Row(hashtags=[], media=None, symbols=[], urls=[], user_mentions=[Row(id=2375699646, id_str='2375699646', indices=[3, 16], name='Haltom City PD', screen_name='HaltomCityPD')]), extended_entities=None, extended_tweet=None, favorite_count=0, favorited=False, filter_level='low', geo=None, id=1529573720297295873, id_str='1529573720297295873', in_reply_to_screen_name=None, in_reply_to_status_id=None, in_reply_to_status_id_str=None, in_reply_to_user_id=None, in_reply_to_user_id_str=None, is_quote_status=False, lang='en', place=None, possibly_sensitive=None, quote_count=0, quoted_status=None, quoted_status_id=None, quoted_status_id_str=None, quoted_status_permalink=None, quoted_text=None, reply_count=0, retweet_count=0, retweeted='RT', retweeted_from='HaltomCityPD', retweeted_status=Row(coordinates=None, created_at='Wed May 25 20:53:25 +0000 2022', display_text_range=None, entities=Row(hashtag

As the tweets after filtering with keywords the count is around **20M** which is not a practical number so wrting a **User defined fucntion** to find the importance of the tweet i.e., whether the text has the word that is in the provided list which assures the tweets are related to education

## Step 4: User defind function to find importance

In [33]:
@F.udf
def get_importance(text):
    words = text.split()
    total_count = 0
    for i in education_keywords:
        occurance_count = words.count(i)
        total_count += occurance_count

    if(total_count > 1):
        return 1
    else:
        return 0

### Filtering based on the importance

In [34]:
df_filtered_education_explicit = df_filtered_education_explicit.withColumn("important", get_importance("stripped"))

In [35]:
df_filtered_education_explicit = df_filtered_education_explicit.filter("important==1")

In [36]:
df_filtered_education_explicit.count()

3141909

In [38]:
pd_df = df_filtered_education_explicit.select('stripped', 'tweet_text').limit(5).cache().toPandas()

In [39]:
import pandas as pd

In [43]:
pd.set_option('display.max_colwidth', 1000)

In [44]:
pd_df

,stripped,tweet_text
0,trump happened because an antiquated amp anti-democratic electoral college defied the will of the people bush happened for the same reason the 2 worst gop presidents in recent history were both the products of the electoral college - and democrats still won’t campaign against it,trump happened because an antiquated &amp; anti-democratic electoral college defied the will of the people. bush happened for the same reason. the 2 worst gop presidents in recent history were both the products of the electoral college - and democrats still won’t campaign against it.
1,a childs ability to read is a key indicator of the likelihood they will graduate high school further evidence connects low literacy with the likelihood an individual could end up in prison — and keep returning\n,a child's ability to read is a key indicator of the likelihood they will graduate high school. further evidence connects low literacy with the likelihood an individual could end up in prison — and keep returning.\nhttps://t.co/c7pje6plnr
2,when i graduated high school my mama went to a rent-to-own store and got a computer so i could go to college with a laptop over the course of her payments she ended up paying 2400 for a 700 laptop being poor is expensive amp college isn’t the equalizer we think it is,when i graduated high school my mama went to a rent-to-own store and got a computer so i could go to college with a laptop. over the course of her payments she ended up paying $2400 for a $700 laptop. being poor is expensive &amp; college isn’t the equalizer we think it is.
3,when i graduated high school my mama went to a rent-to-own store and got a computer so i could go to college with a laptop over the course of her payments she ended up paying 2400 for a 700 laptop being poor is expensive amp college isn’t the equalizer we think it is,when i graduated high school my mama went to a rent-to-own store and got a computer so i could go to college with a laptop. over the course of her payments she ended up paying $2400 for a $700 laptop. being poor is expensive &amp; college isn’t the equalizer we think it is.
4,agtg after a few great conversations with coach adamries5 i’m blessed to have earned an offer to continue my education and play football at knox college boscofootball fb_knoxcollege allegianceol chriswardol kodyafusia coachfredy nlprospect coach_jjhall,"#agtg after a few great conversations with coach @adamries5, i’m blessed to have earned an offer to continue my education and play football at knox college. @boscofootball @fb_knoxcollege @allegianceol @chriswardol @kodyafusia @coachfredy @nlprospect @coach_jjhall https://t.co/qcfguljgo1"


## Step 5: Saving the file to parquet

### Saving the final tweets which is around 3M to the parquet to store intermediate results for faster execution and save resources

In [ ]:
%%time
df_filtered_education_explicit.write.mode('overwrite').parquet("gs://msca-bdp-students-bucket/shared_data/harshamuthukuru/df_filtered_education_explicit1")

CPU times: user 2.74 s, sys: 730 ms, total: 3.47 s
Wall time: 11min 49s


In [ ]:
# df_raw_filtered = spark.read.parquet("gs://msca-bdp-students-bucket/shared_data/harshamuthukuru/raw_df_filtered1")

In [46]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Tue, 07 March 2023 19:41:27'